In [3]:
# Read in the tiny shakespeare dataset

import os 

cwd = os.getcwd()

with open(os.path.join(cwd, 'datasets/input.txt'), 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
# Get a sorted list of all characters appearing in the dataset

chars = sorted(list(set(text)))
vocab_size = len(chars)

In [6]:
print(''.join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [9]:
# Tokenize the data: create a mapping from characters to integers and vice versa
# Create encode/decode functions to apply mapping

s_to_i = { ch:i for i, ch in enumerate(chars)}
i_to_s = { i:ch for i, ch in enumerate(chars)}

encode = lambda s: [s_to_i[c] for c in s]
decode = lambda n: ''.join([i_to_s[i] for i in n])

In [12]:
# Put the tokenized data into a pytorch tensor

import torch
data = torch.tensor(encode(text), dtype=torch.long)

print(data[:100])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [13]:
# Split data into train and validation

n = int(0.9*len(data))

train_data = data[:n]
val_data = data[n:]

In [14]:
# Define function to return the input data for the transformer which will be called on at each training step

block_size = 8
batch_size = 4

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    return x, y

In [21]:
# Define the Bigram Language model

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1336)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C) -- (batch_size, block_size, vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

xb, yb = get_batch('train')

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.6467, grad_fn=<NllLossBackward0>)

c-jMSTt,wOxz&!JBXPyPCxCRFXLunwyp'hM
;'iR,$'
MimUASUltVO&ikB?Y-scZJUFiiFK:RqL:?u;fq,BZ&?RtVCUvPK!Om3t


In [22]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [24]:
batch_size = 32
for steps in range(1000): # increase number of steps for good results... 
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4023187160491943


In [25]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


I in toushthes Pld and te

LBus bus bem it oougrid Jouricin.
RDun; s mor! ay. b AUKIZERCan fest grfotholf beenn'deeve n be mooura the d ben hy med iof venistant!
s elonno te'sorr iqu.
IO s s, anghnouny, my be t wr owir twnst kisprd man yol tis alee we ive m!
TRDWAng, ath
Fre if'lor:
IOMETofeakeve it my

NCole min narito ra wan se, ban:
LAhend t, RIFont Hoghad my Re RI sst? t
AS:
NIO:
QUphaws be,

Houthatheal cceaver?
FLI g
EThaldowiberdseyeal;
AS:
Pononde Juste,

As d kneanthoury: otherckean mer
